In [28]:
from data_parser import VelParser
from entropy_compressor import ZCompressor
from quantizator import Quantizator
from wavelet_compressor import WaveletCompressor

import os
import shutil
import numpy as np
from typing import Dict, List
from pathlib import Path
from pprint import pprint

COMPRESSED_PATH = 'compressed'


class Compressor:
    def __init__(self):
        if not os.path.exists(COMPRESSED_PATH):
            os.mkdir(COMPRESSED_PATH)
        self.entropy_compressor = ZCompressor()
        self.quantizator = Quantizator()
        self.wavelet_compressor = WaveletCompressor()
        self.parser = VelParser()
    
    def total_compress(self, data: np.ndarray) -> List[np.ndarray | np.float64]:
        transformed = self.wavelet_compressor.wavelet_transform(data)
        details = str(transformed[1:])
        wavelet_compressed = transformed[0]
        quantized, q_step, min_val = self.quantizator.uniform_quantization(wavelet_compressed)
        entropy_compressed = self.entropy_compressor.compress(quantized).hex()
        return [entropy_compressed, q_step, min_val, details]

    def save_mesh(self, mesh_data: Dict[str, np.ndarray]) -> None:
        # faces, boundary, neighbour, owner, pts = mesh_data.values()
        with open(COMPRESSED_PATH + '/' + 'mesh', mode='w+') as f:
            for (key, value) in list(mesh_data.items())[:-1]:
                compressed_ndarray, q_step, min_val, details = self.total_compress(value)
                f.write(f'{q_step} {min_val}\n')
                f.write(str(compressed_ndarray) + '\n')
                f.write(str(details) + '\n')

    def compress_and_save(self, model: str, base_path: str, dim: str):
        vels = list(map(str, sorted(self.parser.get_vels(
            model=model,
            base_path=base_path,
            dimpath=dim,
        ), key=lambda x: str(x))))
        mesh_data = self.parser.get_mesh_data(
            path_to_vel=vels[0],
        )
        self.save_mesh(mesh_data=mesh_data)

        for vel in vels:
            vellast = vel.split('/')[-1]
            data = self.parser.get_vel_data(
                vel=vellast,
                base_path=base_path,
                model=model,
                dimpath=dim,
            )
            with open(COMPRESSED_PATH + '/' + vellast, mode='w+') as f:
                times = list(data.keys())
                f.write(f'{";".join(times)}\n')
                for (_, value) in list(data.items())[1:]:
                    p, t, u, rho = value
                    compressed_p, p_q_step, p_min_val, details_p = self.total_compress(p)
                    compressed_t, t_q_step, t_min_val, details_t = self.total_compress(t)
                    compressed_u, u_q_step, u_min_val, details_u = self.total_compress(u)
                    compressed_rho, rho_q_step, rho_min_val, details_rho = self.total_compress(rho)
                    f.write(f'{p_q_step} {p_min_val}\n')
                    f.write(f'{str(compressed_p)}' + '\n')
                    f.write(f'{str(details_p)}' + '\n')

                    f.write(f'{t_q_step} {t_min_val}\n')
                    f.write(f'{str(compressed_t)}' + '\n')
                    f.write(f'{str(details_t)}' + '\n')

                    f.write(f'{u_q_step} {u_min_val}\n')
                    f.write(f'{str(compressed_u)}' + '\n')
                    f.write(f'{str(details_u)}' + '\n')

                    f.write(f'{rho_q_step} {rho_min_val}\n')
                    f.write(f'{str(compressed_rho)}' + '\n')
                    f.write(f'{str(details_rho)}' + '\n')

In [29]:
def doall(
    model: str,
    base_path: str,
    dim: str,
):
    compressor = Compressor()
    compressor.compress_and_save(
        model=model,
        base_path=base_path,
        dim=dim,
    )
    vels = list(map(str, sorted(compressor.parser.get_vels(
        model='data_wage',
        base_path='data',
        dimpath='low_dim',
    ), key=lambda x: str(x))))
    mesh_data = compressor.parser.get_mesh_data(
        path_to_vel=vels[0],
    )
    compressor.save_mesh(mesh_data=mesh_data)

In [30]:
doall(
    model='data_wage',
    base_path='data',
    dim='low_dim'
)

data/data_wage/low_dim/vel3.0
[PosixPath('data/data_wage/low_dim/vel3.0/0'), PosixPath('data/data_wage/low_dim/vel3.0/0.02'), PosixPath('data/data_wage/low_dim/vel3.0/0.04'), PosixPath('data/data_wage/low_dim/vel3.0/0.06'), PosixPath('data/data_wage/low_dim/vel3.0/0.08'), PosixPath('data/data_wage/low_dim/vel3.0/0.1'), PosixPath('data/data_wage/low_dim/vel3.0/0.12'), PosixPath('data/data_wage/low_dim/vel3.0/0.14'), PosixPath('data/data_wage/low_dim/vel3.0/0.16'), PosixPath('data/data_wage/low_dim/vel3.0/0.18'), PosixPath('data/data_wage/low_dim/vel3.0/0.2'), PosixPath('data/data_wage/low_dim/vel3.0/constant'), PosixPath('data/data_wage/low_dim/vel3.0/system')]
[PosixPath('data/data_wage/low_dim/vel3.0/0.02'), PosixPath('data/data_wage/low_dim/vel3.0/0.04'), PosixPath('data/data_wage/low_dim/vel3.0/0.06'), PosixPath('data/data_wage/low_dim/vel3.0/0.08'), PosixPath('data/data_wage/low_dim/vel3.0/0.12'), PosixPath('data/data_wage/low_dim/vel3.0/0.14'), PosixPath('data/data_wage/low_dim/ve